# CampaignSense — Ingestão & Preparação de Dados

Este notebook estabelece a **base técnica e contratual de dados** da POC **CampaignSense**, cobrindo as etapas de **auditoria, preparação e estruturação inicial da base de clientes**.

O escopo é restrito à qualidade, coerência e organização dos dados, criando as condições necessárias para as etapas subsequentes de **análise exploratória orientada à decisão**, **modelagem de propensão** e **profit targeting**.

---

## Contexto analítico

Campanhas de marketing operam sob restrições de orçamento e capacidade operacional, o que exige **priorização de clientes com maior valor esperado**, em vez de contato indiscriminado da base.

No contexto da POC **CampaignSense**, os dados são tratados como uma **base estática de clientes**, na qual cada registro representa o estado consolidado de um cliente no momento da campanha. O dataset público de marketing é utilizado como **proxy de um cenário real de CRM Analytics**, permitindo simular decisões de priorização orientadas a retorno financeiro.

Este notebook define a **camada fundacional da POC**, sendo responsável por:

* auditar a qualidade estrutural e a integridade dos dados;
* aplicar decisões explícitas de preparação, tipagem e higienização;
* organizar as variáveis em grupos coerentes de features;
* definir e executar uma estratégia de particionamento alinhada ao problema de negócio.

> Para a descrição detalhada das variáveis disponíveis, consultar o [**Dicionário de Dados**](`references/01_dicionario_de_dados.md`).

---

## Objetivo do notebook

* Auditar a estrutura e a qualidade do dataset original;
* Aplicar decisões explícitas de tipagem, higienização e tratamento de inconsistências;
* Definir o contrato de variáveis e criar derivações mínimas necessárias ao contexto de CRM;
* Executar um **particionamento estratificado** dos dados, preservando a taxa de resposta;
* Garantir que os dados utilizados nas etapas subsequentes sejam **reprodutíveis, auditáveis e consistentes**.

---

## Entregáveis

Ao final deste notebook, são gerados os seguintes artefatos:

* Conjuntos de dados particionados:

  * `train`
  * `valid`
  * `test`
* Arquivo de metadados do particionamento, contendo:

  * dimensões de cada split;
  * taxa de resposta por conjunto;
  * parâmetros e decisões aplicadas no processo.
* Base de dados preparada para reutilização direta nos notebooks de **EDA**, **modelagem** e **decisão** da POC CampaignSense.

## 1. Setup e carregamento dos dados

In [1]:
# Governança: habilita import de módulos do projeto a partir do diretório /notebooks
import sys
sys.path.insert(0, "..")

# Standard library
import json
from datetime import datetime

# Third-party
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Local
from src.paths import (
    RAW_DATA,
    PROCESSED_TRAIN,
    PROCESSED_VALID,
    PROCESSED_TEST,
    PROCESSED_SPLIT_METADATA,
)

# Governança: seed global para reprodutibilidade do notebook
SEED = 42
np.random.seed(SEED)

# Configuração de exibição (não altera a persistência dos dados)
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.4f}".format)

In [2]:
# Ingestão: dataset bruto (camada raw)
df = pd.read_csv(
    RAW_DATA,
    sep="\t",
    encoding="utf-8",
)

display(df.head(3))

print(f"Formato inicial do dataset: {df.shape[0]} linhas x {df.shape[1]} colunas")

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0000,0,0,04-09-2012,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0000,1,1,08-03-2014,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0000,0,0,21-08-2013,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0


Formato inicial do dataset: 2240 linhas x 29 colunas


In [3]:
# Visão geral(tipagem inicial)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

## 2. Definição de Schema e Checagens Básicas de Qualidade

In [4]:
# Schema de referência (colunas-chave)
ID_COL = "ID"
TARGET_COL = "Response"
DATE_COL = "Dt_Customer"

# Critérios mínimos: presença de colunas
assert ID_COL in df.columns, f"Missing required column: {ID_COL}"
assert TARGET_COL in df.columns, f"Missing required column: {TARGET_COL}"
assert DATE_COL in df.columns, f"Missing required column: {DATE_COL}"

# Regra de integridade do target: domínio binário {0,1} (ignorando NA)
target_domain = set(df[TARGET_COL].dropna().unique())
assert target_domain.issubset({0, 1}), f"Invalid target values in {TARGET_COL}: {sorted(target_domain)}"

In [5]:
# Duplicidade: registro completo e grão por cliente (ID)
dup_rows = int(df.duplicated().sum())
dup_ids = int(df[ID_COL].duplicated().sum())

print(
    "[Data Contracts & Quality Checks]\n"
    f"- Duplicated rows: {dup_rows}\n"
    f"- Duplicated {ID_COL}: {dup_ids}"
)

[Data Contracts & Quality Checks]
- Duplicated rows: 0
- Duplicated ID: 0


In [6]:
# Missingness (top 10)
missing_rate = df.isna().mean().sort_values(ascending=False)
display(missing_rate.head(10))

Income           0.0107
ID               0.0000
Year_Birth       0.0000
Education        0.0000
Marital_Status   0.0000
Kidhome          0.0000
Teenhome         0.0000
Dt_Customer      0.0000
Recency          0.0000
MntWines         0.0000
dtype: float64

In [7]:
# Distribuição do target (contagem e proporção)
display(df[TARGET_COL].value_counts(dropna=False))
display(df[TARGET_COL].value_counts(normalize=True, dropna=False))

Response
0    1906
1     334
Name: count, dtype: int64

Response
0   0.8509
1   0.1491
Name: proportion, dtype: float64

### Registro de qualidade e integridade estrutura

A base atende aos **regars estruturais mínimos** definidos para a POC (presença de `ID`, `Response` e `Dt_Customer`).
Não foram observadas duplicidades por linha completa nem por identificador (`ID`), preservando o grão esperado de **1 registro por cliente**.

A taxa de valores ausentes é concentrada em `Income` (~1,07%). Essa característica é registrada como baseline para aplicação de uma política explícita de tratamento na etapa de preparação.

A variável alvo (`Response`) apresenta taxa de resposta em torno de **15%**, caracterizando uma base desbalanceada. Essa condição é incorporada como premissa técnica para as etapas subsequentes, incluindo **particionamento estratificado** e avaliação consistente com o objetivo de priorização orientada a valor.

## 3. Tipagem e Saneamento Inicial

In [8]:
# Regra de tipagem: data de referência do cliente
df["Dt_Customer"] = pd.to_datetime(
    df["Dt_Customer"],
    format="%d-%m-%Y",
    errors="raise",
)
assert df["Dt_Customer"].notna().all(), "Falha na conversão de Dt_Customer para datetime"

In [9]:
# Regra de missingness: Income -> imputação por mediana global
income_median = df["Income"].median()
df["Income"] = df["Income"].fillna(income_median)
assert df["Income"].isna().sum() == 0, "Income permanece com valores ausentes após imputação"

In [10]:
# Regra de contenção: cap em P99 para outliers extremos de Income
income_p99 = df["Income"].quantile(0.99)
df["Income"] = df["Income"].clip(upper=income_p99)

df["Income"].describe()

count    2240.0000
mean    51751.6811
std     20648.6234
min      1730.0000
25%     35538.7500
50%     51381.5000
75%     68289.7500
max     94437.6800
Name: Income, dtype: float64

In [11]:
# Padronização de categóricas: normalização de strings
for col in ["Education", "Marital_Status"]:
    df[col] = df[col].astype(str).str.strip().str.lower()

for col in ["Education", "Marital_Status"]:
    print(col, df[col].unique())

Education ['graduation' 'phd' 'master' 'basic' '2n cycle']
Marital_Status ['single' 'together' 'married' 'divorced' 'widow' 'alone' 'absurd' 'yolo']


### Registro de tipagem e saneamento inicial

A variável `Dt_Customer` foi convertida para o tipo datetime, garantindo **consistência temporal** para derivações e análises subsequentes.

A variável `Income` apresentou taxa residual de valores ausentes (~1%). Foi aplicada **imputação por mediana global**, seguida de **cap em P99** para contenção de outliers extremos. Essas regras visam estabilizar a distribuição sem introduzir transformações estruturais ou inferência.

As variáveis categóricas `Education` e `Marital_Status` foram **padronizadas por normalização de strings** (remoção de espaços e conversão para minúsculas), assegurando **consistência semântica** para EDA e modelagem.

As transformações desta sessão corrigem inconsistências pontuais e **preservam a estrutura original dos dados**, deixando a base pronta para as etapas seguintes de organização de features e particionamento.

## 4. Definição de Grupos de Features e Derivações Mínimas

In [12]:
# Grupos de features
CAT_COLS = ["Education", "Marital_Status"]

NUM_COLS = [
    "Year_Birth", "Income", "Kidhome", "Teenhome", "Recency",
    "MntWines", "MntFruits", "MntMeatProducts", "MntFishProducts",
    "MntSweetProducts", "MntGoldProds",
    "NumDealsPurchases", "NumWebPurchases", "NumCatalogPurchases",
    "NumStorePurchases", "NumWebVisitsMonth",
    "AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5",
    "Complain"
]

In [13]:
# Variáveis técnicas (excluídas de modelagem)
TECH_COLS = ["Z_CostContact", "Z_Revenue"]

# Conjunto base de colunas nesta etapa
BASE_COLS = [ID_COL, TARGET_COL, DATE_COL] + CAT_COLS + NUM_COLS + TECH_COLS

# Verificação estrutural
missing_cols = [c for c in BASE_COLS if c not in df.columns]
assert not missing_cols, f"Colunas esperadas ausentes: {missing_cols}"

In [14]:
# Derivações mínimas (baseadas em informações existentes)
ref_date = df[DATE_COL].max()
ref_year = ref_date.year

df["Age"] = ref_year - df["Year_Birth"]
df["TenureDays"] = (ref_date - df[DATE_COL]).dt.days

# Checagens básicas de integridade
assert (df["Age"] > 0).all(), "Valores inválidos de Age (<= 0)"
assert (df["TenureDays"] >= 0).all(), "Valores inválidos de TenureDays (< 0)"

In [15]:
# Regra de higienização semântica: outliers biológicos
AGE_MAX = 100
n_before = df.shape[0]

df = df[df["Age"] <= AGE_MAX]

n_after = df.shape[0]
print(f"Registros removidos por Age > {AGE_MAX}: {n_before - n_after}")

df[["Age", "TenureDays"]].describe()

Registros removidos por Age > 100: 3


,Age,TenureDays
count,2237.0000,2237.0000
mean,45.0983,353.7903
std,11.7019,202.1380
min,18.0000,0.0000
25%,37.0000,181.0000
50%,44.0000,356.0000
75%,55.0000,529.0000
max,74.0000,699.0000


### Registro de organização de features e derivações mínimas

As variáveis foram organizadas em **grupos explícitos** de features categóricas, numéricas e técnicas. As variáveis técnicas foram mantidas no dataset por rastreabilidade, com exclusão prevista nas etapas de modelagem.

Foram criadas duas derivações mínimas, `Age` e `TenureDays`, a partir de informações já disponíveis, com o objetivo de capturar características demográficas e de relacionamento do cliente **sem introduzir complexidade adicional**.

Durante a derivação de `Age`, foram identificados registros com valores biologicamente implausíveis (`Age > 100`). Esses registros foram tratados como **inconsistências semânticas** e removidos da base, por não representarem clientes válidos no contexto de CRM Analytics.

Após a aplicação dessas regras, a base mantém **integridade estrutural e semântica**, ficando pronta para o particionamento e as etapas subsequentes do pipeline.

## 5. Estratégia e Execução do Split dos Dados

A base é tratada como **estática de clientes**, com 1 registro por cliente no contexto da campanha.
O particionamento em **treino**, **validação** e **teste** é realizado via **amostragem aleatória estratificada por `Response`**, preservando a taxa de resposta entre os conjuntos.

Premissas e garantias desta etapa:

* comparabilidade de base rate entre splits;
* ausência de sobreposição de identificadores entre conjuntos;
* parametrização controlada por seed para reprodutibilidade.

### Parâmetros do split

* `test_size = 0.20`
* `valid_size = 0.20` do conjunto remanescente (≈ 0.16 do total)
* Estratificação por `Response`
* Seed fixa

In [16]:
TEST_SIZE = 0.20
VALID_SIZE_OF_REMAINING = 0.20

# Dataset final desta etapa (inclui derivações mínimas)
FINAL_COLS = BASE_COLS + ["Age", "TenureDays"]
df_final = df[FINAL_COLS].copy()

In [17]:
# Split 1: train_temp vs test (estratificado por Response)
train_temp, test = train_test_split(
    df_final,
    test_size=TEST_SIZE,
    random_state=SEED,
    stratify=df_final[TARGET_COL],
)

# Split 2: train vs valid (estratificado por Response)
train, valid = train_test_split(
    train_temp,
    test_size=VALID_SIZE_OF_REMAINING,
    random_state=SEED,
    stratify=train_temp[TARGET_COL],
)

In [18]:
# Sumário dos splits (dimensão e base rate)
split_summary = pd.DataFrame({
    "rows": {
        "train": int(train.shape[0]),
        "valid": int(valid.shape[0]),
        "test":  int(test.shape[0]),
    },
    "response_rate": {
        "train": float(train[TARGET_COL].mean()),
        "valid": float(valid[TARGET_COL].mean()),
        "test":  float(test[TARGET_COL].mean()),
    }
})
split_summary.style.format({"response_rate": "{:.2%}"})

,rows,response_rate
train,1431,14.95%
valid,358,14.80%
test,448,14.96%


In [19]:
# Integridade: IDs não podem se repetir entre splits
train_ids = set(train[ID_COL])
valid_ids = set(valid[ID_COL])
test_ids  = set(test[ID_COL])

assert train_ids.isdisjoint(valid_ids), "ID overlap: train vs valid"
assert train_ids.isdisjoint(test_ids), "ID overlap: train vs test"
assert valid_ids.isdisjoint(test_ids), "ID overlap: valid vs test"

In [20]:
# Metadados do split (persistência na etapa de exportação)
metadata = {
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "seed": int(SEED),
    "strategy": "stratified_random",
    "target_col": TARGET_COL,
    "id_col": ID_COL,
    "sizes": {
        "total_rows": int(df_final.shape[0]),
        "train_rows": int(train.shape[0]),
        "valid_rows": int(valid.shape[0]),
        "test_rows": int(test.shape[0]),
        "test_size": float(TEST_SIZE),
        "valid_size_of_remaining": float(VALID_SIZE_OF_REMAINING),
    },
    "base_rate": {
        "train": float(train[TARGET_COL].mean()),
        "valid": float(valid[TARGET_COL].mean()),
        "test":  float(test[TARGET_COL].mean()),
    },
    "data_prep": {
        "income": {"imputation": "median_global", "capping": "p99"},
        "ref_date": str(df[DATE_COL].max().date()),
    },
}

## 6. Exportação dos Artefatos

Os conjuntos particionados (**`train`**, **`valid`** e **`test`**) são persistidos na camada `data/processed/`.

Adicionalmente, é gerado um arquivo de **metadados do particionamento**, registrando parâmetros, tamanhos e taxas de resposta.

Esses artefatos constituem o **estado de referência** da POC ao final da preparação, sendo reutilizados diretamente nos notebooks subsequentes.

In [21]:
# Persistência dos datasets particionados
train.to_parquet(PROCESSED_TRAIN, index=False)
valid.to_parquet(PROCESSED_VALID, index=False)
test.to_parquet(PROCESSED_TEST, index=False)

# Persistência do metadata do split (JSON)
with open(PROCESSED_SPLIT_METADATA, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

## Encerramento

Este notebook consolida a **base técnica e operacional da POC CampaignSense**.

Ao seu término, a POC dispõe de:

* um dataset auditado e estruturalmente consistente;
* schema, variável alvo e regras estruturais explicitamente definidos;
* inconsistências pontuais tratadas por meio de tipagem, higienização e contenção controlada de outliers;
* variáveis organizadas em grupos semânticos claros, com derivações mínimas aplicadas;
* particionamento estratificado em treino, validação e teste, preservando a taxa de resposta;
* artefatos persistidos (datasets e metadados), assegurando **reprodutibilidade e rastreabilidade**.

Esse conjunto constitui a **referência única de dados** para as etapas subsequentes de **EDA orientada à decisão**, **modelagem de propensão** e **profit targeting**.